In [69]:
import sqlite3
import pandas as pd

# Create your connection.
from twitter.nlp_util import process_tweet

cnx = sqlite3.connect('db.sqlite3')

df = pd.read_sql_query(
    "SELECT id, conversation_id, created_at, text, author_id,in_reply_to_user_id FROM delab_timeline WHERE lang='en'",
    cnx)

df.head(3)


,id,conversation_id,created_at,text,author_id,in_reply_to_user_id
0,1,1435703745304612870,2021-09-08 20:37:01,RT @OregonOEM: 🚩🌩🔥 Red Flag and #FireWeather w...,14838508,NaN
1,2,1435664738353098756,2021-09-08 18:02:01,It's #NationalPreparednessMonth. Help ensure t...,14838508,NaN
2,3,1435663883595706370,2021-09-08 17:58:37,RT @OregonOEM: Oregon is still recovering from...,14838508,NaN


Now we are importing the libraries.

The first idea was to follow lda https://radimrehurek.com/gensim/auto_examples/tutorials/run_lda.html

but turns out it is to little information

In [70]:
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

import spacy
from spacy.lang.en.stop_words import STOP_WORDS

from tqdm import tqdm_notebook as tqdm
from pprint import pprint

nlp = spacy.load("en_core_web_lg")

Showing some descriptive statistics for the author corpus downloaded from twitter.

In [72]:
df.author_id.nunique() # the number of unique authors in the dataset

69

In [74]:
df_reduced = df[["author_id", "text", "id"]]
#df_reduced = df_reduced.groupby('author_id')
# df_reduced.count()

df_reshaped = df_reduced.pivot(index="id",columns="author_id", values="text")
mask = 400 > df_reshaped.nunique()
mask = mask[mask == True]
df_reshaped.drop(columns=mask.index,inplace=True)
df_reshaped.nunique() # the number of tweets of the authors that have more then 400 tweets

author_id
16558158               478
18616003               946
26998226               469
382814447              446
1005470991668084736    445
1106611172462219265    491
1162371171805011968    447
1239172010363826183    427
1292908140975943681    414
1402252385427222528    414
1403930956428460035    441
dtype: int64

The following takes the pandas dataframe and converts it to a dictionary with the author ids as keys and the twitter
corpora as values.

In [82]:
#df_reshaped.shape
author_corpora = df_reshaped.to_dict(orient="series")
for author_id, tweets in author_corpora.items():
    author_corpora[author_id] = tweets.dropna()

example_corpus = author_corpora[next(iter(author_corpora))]
example_corpus

id
8247                                   @ncreen_same ouch!
8248    That last tweet got some responses from spambo...
8249    'virtually' virtually means: real\n\nSo it sho...
8250                               Sounds great on paper!
8251    Any .com.au registrar recommendations? So far ...
                              ...                        
8742    RT @paydirtapp: Check out our Free Invoice Cre...
8743    @taitems @mmilo yeah nice site, suggestion: ht...
8744    RT @MichaelFHansen: Zendesk eyes Southeast Asi...
8745    While LinkedIn has been changing drastically o...
8746    Another LinkedIn email fail… now they want me ...
Name: 16558158, Length: 478, dtype: object

Now we are ready to analyze the corpora of the authors tweets. We do one example with the first corpus


In [113]:
docs = []

for tweet in example_corpus:
    docs.append(process_tweet(tweet))

# Compute bigrams.
from gensim.models import Phrases

# Add bigrams and trigrams to docs (only ones that appear 20 times or more).
bigram = Phrases(docs, min_count=20)
for idx in range(len(docs)):
    for token in bigram[docs[idx]]:
        if '_' in token:
            # Token is a bigram, add to document.
            docs[idx].append(token)

###############################################################################
# We remove rare words and common words based on their *document frequency*.
# Below we remove words that appear in less than 20 documents or in more than
# 50% of the documents. Consider trying to remove words only based on their
# frequency, or maybe combining that with this approach.
#

# Remove rare and common tokens.
from gensim.corpora import Dictionary

# Create a dictionary representation of the documents.
dictionary = Dictionary(docs)

# Filter out words that occur less than 20 documents, or more than 50% of the documents.
dictionary.filter_extremes(no_below=20, no_above=0.5)

###############################################################################
# Finally, we transform the documents to a vectorized form. We simply compute
# the frequency of each word, including the bigrams.
#

# Bag-of-words representation of the documents.
corpus = [dictionary.doc2bow(doc) for doc in docs]

###############################################################################
# Let's see how many tokens and documents we have to train on.
#

print('Number of unique tokens: %d' % len(dictionary))
print('Number of documents: %d' % len(corpus))

# Train LDA model.
from gensim.models import LdaModel

# Set training parameters.
num_topics = 3
chunksize = 2000
passes = 20
iterations = 400
eval_every = None  # Don't evaluate model perplexity, takes too much time.

# Make a index to word dictionary.
temp = dictionary[0]  # This is only to "load" the dictionary.
id2word = dictionary.id2token

model = LdaModel(
    corpus=corpus,
    id2word=id2word,
    chunksize=chunksize,
    alpha='auto',
    eta='auto',
    iterations=iterations,
    num_topics=num_topics,
    passes=passes,
    eval_every=eval_every
)

# Note that we use the "Umass" topic coherence measure here (see
# :py:func:`gensim.models.ldamodel.LdaModel.top_topics`), Gensim has recently
# obtained an implementation of the "AKSW" topic coherence measure (see
# accompanying blog post, http://rare-technologies.com/what-is-topic-coherence/).

top_topics = model.top_topics(corpus) #, num_words=20)

# Average topic coherence is the sum of topic coherences of all topics, divided by the number of topics.
avg_topic_coherence = sum([t[1] for t in top_topics]) / num_topics
print('Average topic coherence: %.4f.' % avg_topic_coherence)

from pprint import pprint
pprint(top_topics)

Number of unique tokens: 9
Number of documents: 478
Average topic coherence: -5.7811.
[([(0.3684394, 'like'),
   (0.31433037, 'know'),
   (0.28787804, 'use'),
   (0.0053321696, 'look'),
   (0.0049920008, 'new'),
   (0.0049395612, 'get'),
   (0.0047678538, 'work'),
   (0.004673165, '…'),
   (0.0046474542, '...')],
  -5.77066551539591),
 ([(0.2638249, '…'),
   (0.2637392, 'work'),
   (0.24274658, 'get'),
   (0.21125293, 'new'),
   (0.0038828384, 'know'),
   (0.0037253946, 'use'),
   (0.0036451528, 'like'),
   (0.0036288158, 'look'),
   (0.00355417, '...')],
  -5.7780273587207835),
 ([(0.52524346, '...'),
   (0.42429098, 'look'),
   (0.0077454015, 'like'),
   (0.0073940223, 'new'),
   (0.0073653413, 'know'),
   (0.007064507, 'get'),
   (0.0070528523, 'work'),
   (0.0070286235, '…'),
   (0.006814815, 'use')],
  -5.794644230029977)]


As this resulted in disappointed non-nouns as topics the alternative would be to use the named entities directly


In [107]:
found_entities = set()

for tweet in example_corpus:
    doc = nlp(tweet)
    sentences = list(doc.sents)
    for sentence in sentences:
        if sentence.ents:
            found_entities.add(str(sentence.ents))

pprint(found_entities)


{'["Thinking with Type]',
 '[#Melbourne #, 2013]',
 '[#]',
 '[#susanalbumparty #craftbeer #, melbourne]',
 '[#wuws #]',
 '[0, http://t.co/K8cPzW6qPk]',
 '[1 to 10]',
 '[1.30, 1.26]',
 '[100%, first]',
 '[100, a few seconds]',
 '[1000th]',
 '[10B+]',
 '[11000 hours]',
 '[140]',
 '[16 days]',
 '[1]',
 '[1st]',
 '[20, first]',
 '[234272]',
 '[2]',
 '[3, Tiiim]',
 '[35%]',
 '[3]',
 '[3rd]',
 '[4]',
 '[7 minutes]',
 '[700m]',
 '[7487944]',
 '[75]',
 '[8]',
 '[@ArondeParon, Redmine]',
 '[@BrewSmithAu]',
 '[@BugHerd]',
 '[@FLTBTG http://t.co/y9LJeAehCV]',
 '[@GVRV &lt;/genuinelywondering&gt]',
 '[@GVRV]',
 '[@KeyPayApp]',
 '[@Moocar]',
 '[@Panviva]',
 '[@Rackspace]',
 '[@TomPisel]',
 '[@UXMastery]',
 '[@Uber_Sydney, today]',
 '[@alandownie @bigyahu, 10%, the last month]',
 '[@alandownie @ivanmelvin]',
 '[@alandownie @mmilo]',
 '[@alandownie @ncreen_same @therealdevgeeks]',
 '[@alandownie @ncreen_same]',
 '[@alandownie @rioter @ncreen_same]',
 '[@alandownie @rioter]',
 '[@alandownie @skwashd]'